# Introducción a [Apache Spark](https://spark.apache.org/), curso 25-26

Objetivo del notebook: crear una SparkSession, entender la arquitectura básica (Driver/Ejecutors), diferenciar API estructurada (DataFrames) vs RDDs, y ejecutar transformaciones/acciones con explicación del plan de ejecución.

![Spark](https://upload.wikimedia.org/wikipedia/commons/f/f3/Apache_Spark_logo.svg)


## Plataforma de computación cluster rápida

- Extiende el modelo MapReduce y soporta eficientemente otros tipos de computación:
  - Queries interactivas (SQL/DataFrames)
  - Procesado streaming (*Structured Streaming*)
- Soporta computaciones en memoria y optimizaciones a nivel de motor (Catalyst + Tungsten)
- Mejora frente a MapReduce clásico para cargas complejas (aceleración significativa en ETL e interactivo; el factor depende del caso)

### Propósito general

- Modos de funcionamiento: batch, interactivo y streaming
- Reduce el número de herramientas a emplear y mantener

Modos de despliegue habituales: `local[*]`, Standalone, YARN, Kubernetes.

### Historia

- Iniciado en 2009 en el UC Berkeley RAD Lab (AMPLab)
  - Motivado por la ineficiencia de MapReduce para trabajos iterativos e interactivos
- Mayores contribuidores: [Databricks](https://databricks.com/), Yahoo! e Intel
- Declarado open source en marzo de 2010
- Transferido a la Apache Software Foundation en junio de 2013, TLP en febrero de 2014
- Uno de los proyectos Big Data más activos
- Versión 1.0 lanzada en mayo de 2014

Hoy (Spark 4.x) incorpora mejoras como Adaptive Query Execution (AQE), Pandas UDFs mejoradas, ANSI SQL mode y optimizaciones en el planificador. Notas: https://spark.apache.org/releases/

#### Características de Spark

- Soporta gran variedad de workloads: batch, queries interactivas (SQL/DataFrames), streaming (Structured Streaming), machine learning (MLlib), y grafos (GraphFrames en el ecosistema)
- APIs en Scala, Java, Python, SQL y R
- Shells interactivos en Scala, Python, SQL y R
- Se integra con otras soluciones Big Data: HDFS, S3, Kafka, Cassandra, etc.

### La pila Spark
![sparkstack](http://persoal.citius.usc.es/tf.pena/TCDM/figs/sparkstack.png)

(Fuente: H. Karau, A. Konwinski, P. Wendell, M. Zaharia, "Learning Spark", O'Reilly, 2015)

## APIs del Spark Core
Spark ofrece dos APIs:

- API estructurada o de alto nivel (SQL/DataFrames/Datasets)
- API de bajo nivel (RDD)

Cada API ofrece diferentes tipos de datos y trade-offs:

- Se recomienda usar la API estructurada por su mayor rendimiento y expresividad SQL
- La API de bajo nivel permite mayor control sobre particionado y algoritmos custom
- La API de alto nivel utiliza internamente primitivas de bajo nivel

## Tipos de datos en la API estructurada

### Datasets
Colección distribuida de objetos del mismo tipo (tipados)

- Introducida en Spark ≥ 1.6
- Disponible en Scala y Java (no en Python/R por tipado dinámico)

### DataFrames
Colección distribuida organizada en columnas con nombre

- Conceptualmente: tabla relacional / pandas DataFrame
- Disponible en Scala, Java, Python y R
- En Java/Scala: un DataFrame es un Dataset[Row]

## Tipos de datos en la API de bajo nivel
### RDDs (Resilient Distributed Datasets)

- Lista distribuida de objetos; API base de Spark 1.x
- Útiles cuando se requiere control fino (particiones, operaciones no expresables en SQL)

## Mejor rendimiento de la API estructurada

- DataFrames/Datasets aprovechan Catalyst (optimizador de consultas) y Tungsten (motor de ejecución)

<img src="https://databricks.com/wp-content/uploads/2015/02/Screen-Shot-2015-02-16-at-9.46.39-AM.png" alt="Mejora de rendimiento" style="width: 650px;"/>

Fuente: [Recent performance improvements in Apache Spark: SQL, Python, DataFrames, and More](https://databricks.com/blog/2015/04/24/recent-performance-improvements-in-apache-spark-sql-python-dataframes-and-more.html)

### ¿Cuándo usar cada API?
- DataFrame (por defecto): ETL, SQL, agregaciones, joins; mejor rendimiento
- RDD: lógica no expresable en SQL, control de particiones, algoritmos personalizados

## Conceptos clave
![sparkcontext](http://persoal.citius.usc.es/tf.pena/TCDM/figs/sparkcontext.png)

Términos básicos:
- Job: conjunto de stages disparado por una acción
- Stage: grupo de tareas sin shuffle entre ellas
- Task: unidad de trabajo sobre una partición
- Partición: porción del dataset procesada en paralelo
- DAG: grafo acíclico de transformaciones (evaluación perezosa)
- Dependencias: narrow (sin shuffle) vs wide (con shuffle)

(Fuente: H. Karau, A. Konwinski, P. Wendell, M. Zaharia, "Learning Spark", O'Reilly, 2015)

#### Driver

- Crea un `SparkContext`
- Convierte el programa de usuario en tareas:
  - `DAG` de operaciones lógico -> plan de ejecución físico
- Planifica las tareas en los ejecutores

#### SparkSession y SparkContext

- `SparkSession`: punto de entrada de todas las funcionalidades de Spark
  - Permite especificar la configuración de la aplicación Spark
  - En notebooks/shell puede existir ya creada (variable `spark`)
- `SparkContext`: realiza la conexión con el cluster
  - Se obtiene desde `SparkSession` (`spark.sparkContext`)
  - Punto de entrada para la API de bajo nivel
  - En el notebook (o shell), puede estar predefinida como `sc`

- Creación en un script Python

In [ ]:
%pip install -q pyspark

import sys

import pyspark

print("PySpark", pyspark.__version__, "con Python", sys.version.split()[0])

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Crear u obtener una SparkSession (en local). En clúster, no fijar master aquí.
spark: SparkSession = (
    SparkSession.builder.appName("TCDM-IntroduccionSpark-3-01")
    .master("local[*]")  # solo para entorno local/notebook
    .config(
        "spark.sql.shuffle.partitions", "8"
    )  # demo de parámetro: particiones por defecto en local
    .getOrCreate()
)

sc: SparkContext = spark.sparkContext

print("Spark version:", spark.version, "| Master:", sc.master, "| App:", sc.appName)

#### Executors

- Ejecutan las tareas individuales y devuelven los resultados al Driver
- Proporcionan almacenamiento en memoria para los datos de las tareas

#### Cluster Manager

- Componente enchufable en Spark
- Modos típicos: Standalone, YARN, Kubernetes (Mesos en desuso)
- El modo `local[*]` emula ejecución local sin ejecutores separados

## Instalación de Spark

- Para **Scala**:
  1. Descargar Apache Spark de http://spark.apache.org/downloads.html
     - “Pre-built for Hadoop 3.x and later” incorpora binarios compatibles
     - También se puede construir desde el código fuente
  2. Extraer el fichero descargado
- Para **Python (PySpark)**:
  1. Instalar con `pip`: `pip3 install pyspark` (añade JARs y dependencias necesarias. Quizá necesario iniciar un entorno virtual)
  2. (Opcional) Añadir `~/.local/bin` al `PATH` si usas instalación de usuario
  3. (Opcional en clúster) Si accedes a HDFS/YARN: `export HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop`

## Ejecución de Spark
1. Consolas interactivas
    - Scala: `spark-shell`
    - Python: `pyspark`
        - Con IPython: `PYSPARK_DRIVER_PYTHON=ipython pyspark`
        - Con Jupyter: `PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS="notebook" pyspark`
    - R: `sparkR`
    - SQL: `spark-sql`
    - [Apache Zeppelin](https://zeppelin.apache.org/)
2. Script con `spark-submit`
    - Ej.: `spark-submit --master local[4] --name TCDM-Intro app.py`

In [ ]:
# Ejemplo: muestra la versión de PySpark
try:
    print(f"Versión de PySpark {spark.version}.")
except NameError:
    print("SparkSession no inicializada; ejecuta la celda de creación de spark.")

In [ ]:
# Cerrar la sesión de Spark al finalizar
try:
    spark.stop()
    print("SparkSession detenida.")
except Exception as e:
    print("No se pudo detener SparkSession:", e)

## Documentación
La documentación oficial sobre Apache Spark está en https://spark.apache.org/docs/latest/

APIs por lenguaje:

  - Python: https://spark.apache.org/docs/latest/api/python/
  - Scala: https://spark.apache.org/docs/latest/api/scala/
  - Java: https://spark.apache.org/docs/latest/api/java/

Guías útiles:
- SQL programming guide: https://spark.apache.org/docs/latest/sql-programming-guide.html
- Funciones SQL (Python): https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html
- Pandas API on Spark (opcional): https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html